# Score: 98.985%

### Imports

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Data

In [2]:
train = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/train.csv")
test = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/test.csv")

### Processing the data

In [3]:
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

train_y = train['label']
train_x = train.drop(['label'], axis=1)

# Reshape
train_x = train_x.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Normalize
train_x = train_x / 255.0
test = test / 255.0

train_y = to_categorical(train_y, num_classes = 10)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)

Using TensorFlow backend.


### Creating the model

In [4]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

# Compile the model
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics=["accuracy"])

### Generating more images

In [5]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1  # randomly shift images vertically (fraction of total height)
)


data_generator.fit(train_x)

# Fitting the model to the data

### High batch size

In [6]:
epochs = 10
batch_size = 120

history = model.fit_generator(data_generator.flow(train_x, train_y, batch_size=batch_size),
                              epochs = epochs, validation_data = (val_x, val_y))

Epoch 1/10
280/280 [==============================] - 192s 685ms/step - loss: 0.8627 - acc: 0.7124 - val_loss: 0.1630 - val_acc: 0.9506
Epoch 2/10
280/280 [==============================] - 195s 696ms/step - loss: 0.2502 - acc: 0.9276 - val_loss: 0.0714 - val_acc: 0.9782
Epoch 3/10
280/280 [==============================] - 242s 864ms/step - loss: 0.1707 - acc: 0.9521 - val_loss: 0.0673 - val_acc: 0.9801
Epoch 4/10
280/280 [==============================] - 239s 853ms/step - loss: 0.1372 - acc: 0.9600 - val_loss: 0.0453 - val_acc: 0.9856
Epoch 5/10
280/280 [==============================] - 181s 646ms/step - loss: 0.1173 - acc: 0.9668 - val_loss: 0.0436 - val_acc: 0.9881
Epoch 6/10
280/280 [==============================] - 201s 718ms/step - loss: 0.1018 - acc: 0.9714 - val_loss: 0.0390 - val_acc: 0.9893
Epoch 7/10
280/280 [==============================] - 190s 679ms/step - loss: 0.0901 - acc: 0.9748 - val_loss: 0.0367 - val_acc: 0.9899
Epoch 8/10
280/280 [============================

# Predictions

In [7]:
predictions = model.predict(test)

submission = pd.DataFrame({
                "ImageId": range(1,28001),
                "Label": np.argmax(predictions, axis = 1)
                })

# Saving the predictions

In [8]:
submission.to_csv("/Users/pbezuhov/git/Kaggle/submissions/4_large_batch_size.csv", index=False)